In [1]:
import pandas as pd


In [2]:
#data engineering 
df = pd.read_csv('/home/mohammad/Desktop/CHIC/main_models_classes/datasets/Expected Growth in EPS - Next 5 years.csv')  
columns = df.columns  
columns 

Index(['Industry Name', 'Number of Firms', 'CAGR in Net Income- Last 5 years',
       'CAGR in Revenues- Last 5 years',
       'Expected Growth in Revenues - Next 2 years',
       'Expected growth in Revenues - Next 5 years',
       'Expected Growth in EPS - Next 5 years', 'Unnamed: 7', 'Unnamed: 8',
       'Unnamed: 9'],
      dtype='object')

In [3]:
last_2_cols = columns[-2:]  
first_col = columns[0]  
#delete the first column and the last two columns 
df = df.drop([first_col], axis=1) 
df = df.drop(last_2_cols, axis=1) 
#save the new data frame 
df.to_csv('/home/mohammad/Desktop/CHIC/main_models_classes/datasets/Expected Growth in EPS - Next 5 years.csv', index=False)

In [6]:
#check the new data frame 
df = pd.read_csv('/home/mohammad/Desktop/CHIC/main_models_classes/datasets/Expected Growth in EPS - Next 5 years.csv')   
columns = df.columns 
first_col = columns[0] 
last_col = columns[-1]  
df.drop([first_col], axis=1, inplace=True) 
df.drop([last_col], axis=1, inplace=True) 
df.to_csv('/home/mohammad/Desktop/CHIC/main_models_classes/datasets/Expected Growth in EPS - Next 5 years.csv', index=False)

In [7]:
df = pd.read_csv('/home/mohammad/Desktop/CHIC/main_models_classes/datasets/Expected Growth in EPS - Next 5 years.csv') 
columns = df.columns 
df.drop(columns[-2:], axis=1, inplace=True)

In [8]:
df.to_csv('/home/mohammad/Desktop/CHIC/main_models_classes/datasets/Expected Growth in EPS - Next 5 years.csv', index=False)  


In [17]:
def convert_percentage(column):
    if df[column].dtype == 'object':  # Check if the column type is 'object' (strings)
        df[column] = df[column].str.rstrip('%').astype('float') / 100.0

In [42]:
data = pd.read_csv('/home/mohammad/Desktop/CHIC/main_models_classes/datasets/Expected Growth in EPS - Next 5 years.csv') 
columns = data.columns     
feature1 = columns[0]
feature2 = columns[1] 
target = columns[2]   
convert_percentage(feature1)
convert_percentage(feature2)
convert_percentage(target)
X = df[[feature1 , feature2]]
y = df[target] 
X.shape, y.shape

((103, 2), (103,))

In [55]:
import pandas as pd
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.linear_model import SGDRegressor
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import PolynomialFeatures


from sklearn.metrics import mean_squared_error

In [59]:
pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('poly', PolynomialFeatures(degree=2, include_bias=False)),  # Create polynomial features of degree 2
    ('regressor', SGDRegressor(max_iter=10, tol=1e-3, random_state=42))
])   
#drop the missing values 
X.dropna(inplace=True) 
y.dropna(inplace=True) 


/tmp/ipykernel_58065/1317469381.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.dropna(inplace=True)


In [60]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define K-Folds cross-validator for the training set
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Perform cross-validation on the training set
scores = cross_val_score(pipeline, X_train, y_train, cv=kf, scoring='neg_mean_squared_error')

# Convert negative MSE to positive
mse_scores = -scores

# Calculate the mean and standard deviation of the MSE scores
mean_mse = mse_scores.mean()
std_mse = mse_scores.std()

print(f'Mean MSE (cross-validation): {mean_mse}')
print(f'Standard Deviation of MSE (cross-validation): {std_mse}')

# Fit the pipeline on the entire training set and evaluate on the test set
pipeline.fit(X_train, y_train)

# Check the number of iterations taken by the SGDRegressor to converge
iterations = pipeline.named_steps['regressor'].n_iter_
print(f'Number of iterations: {iterations}')

y_pred = pipeline.predict(X_test)
test_mse = mean_squared_error(y_test, y_pred)

print(f'Mean Squared Error on test set: {test_mse}')

Mean MSE (cross-validation): 0.03316973266885691
Standard Deviation of MSE (cross-validation): 0.03820592010485108
Number of iterations: 7
Mean Squared Error on test set: 4.254922722887174
